[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Algebra_Lineal/blob/main/semana8/Problema_del_Viajante.ipynb)

# Introducción

Utilizaremos programación lineal para resolver el **Problema del Viajante** (Travel Salesman Problem).

El problema es encontrar la **ruta más corta que conecta a todas las ciudades**, con las restricciones de que debe ser una única ruta (i.e. no puede haber sub rutas) y sólo se puede entrar y salir de cada ciudad una sola vez.

Se crearán las "coordenadas" de ciudades ficticias y para medir su distancia se usará la distancia euclideana. 

# Dependencias

In [ ]:
!pip install mip
!pip install -U plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

from scipy.spatial import distance_matrix

from itertools import product
from mip import Model, xsum, minimize, BINARY

import plotly.express as px
import plotly.io as pio

In [ ]:
pio.templates.default = "plotly_white"

# Funciones para la visualización de los datos

In [ ]:
def plot_cities(coords):
    fig = px.scatter(coords.reset_index(), 'x', 'y', hover_name='index')
    fig.update_traces(marker=dict(size=15,
                                line=dict(width=2,
                                            color='DarkSlateGrey')),
                    selector=dict(mode='markers')
                    )
    return fig

def plot_cities_and_route(points, edges):
    fig = plot_cities(points)
    for edge in edges:
        fig.add_shape(
            type = 'line', 
            x0 = points.loc[f'ciudad_{edge[0]}', 'x'], 
            x1 = points.loc[f'ciudad_{edge[1]}', 'x'], 
            y0 = points.loc[f'ciudad_{edge[0]}', 'y'], 
            y1 = points.loc[f'ciudad_{edge[1]}', 'y'], 
            line = dict(color = 'rgb(0, 0, 0)'), 
            opacity = .09,
            )
        fig.add_scatter(x = [mid_points[edge[0]][edge[1]][0]], y = [mid_points[edge[0]][edge[1]][1]], text = [c[edge[0]][edge[1]]], mode='text')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='top right')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', showlegend=False)
    fig.show()

In [ ]:
def get_middle_points(coords):
    return [[((x[0]+y[0])/2, (x[1]+y[1])/2) for _, y in coords.iterrows()] for _, x in coords.iterrows()]

# Datos

Primero crearemos las ciudades y sus coordenadas

In [ ]:
np.random.seed(10)

n = 10 # número de ciudades

points = pd.DataFrame(np.random.randint(0, 30, (n, 2)), columns = ['x', 'y'], index = [f'ciudad_{i}' for i in range(n)])  # coordenadas en el plano cartesiano de cada ciudad
points

,x,y
ciudad_0,9,29
ciudad_1,4,15
ciudad_2,0,17
ciudad_3,27,28
ciudad_4,25,29
ciudad_5,16,29
ciudad_6,17,26
ciudad_7,8,9
ciudad_8,0,10
ciudad_9,8,22


Obtenemos el punto medio entre cada ciudad

In [ ]:
mid_points = get_middle_points(points)

In [ ]:
mid_points

[[(9.0, 29.0),
  (6.5, 22.0),
  (4.5, 23.0),
  (18.0, 28.5),
  (17.0, 29.0),
  (12.5, 29.0),
  (13.0, 27.5),
  (8.5, 19.0),
  (4.5, 19.5),
  (8.5, 25.5)],
 [(6.5, 22.0),
  (4.0, 15.0),
  (2.0, 16.0),
  (15.5, 21.5),
  (14.5, 22.0),
  (10.0, 22.0),
  (10.5, 20.5),
  (6.0, 12.0),
  (2.0, 12.5),
  (6.0, 18.5)],
 [(4.5, 23.0),
  (2.0, 16.0),
  (0.0, 17.0),
  (13.5, 22.5),
  (12.5, 23.0),
  (8.0, 23.0),
  (8.5, 21.5),
  (4.0, 13.0),
  (0.0, 13.5),
  (4.0, 19.5)],
 [(18.0, 28.5),
  (15.5, 21.5),
  (13.5, 22.5),
  (27.0, 28.0),
  (26.0, 28.5),
  (21.5, 28.5),
  (22.0, 27.0),
  (17.5, 18.5),
  (13.5, 19.0),
  (17.5, 25.0)],
 [(17.0, 29.0),
  (14.5, 22.0),
  (12.5, 23.0),
  (26.0, 28.5),
  (25.0, 29.0),
  (20.5, 29.0),
  (21.0, 27.5),
  (16.5, 19.0),
  (12.5, 19.5),
  (16.5, 25.5)],
 [(12.5, 29.0),
  (10.0, 22.0),
  (8.0, 23.0),
  (21.5, 28.5),
  (20.5, 29.0),
  (16.0, 29.0),
  (16.5, 27.5),
  (12.0, 19.0),
  (8.0, 19.5),
  (12.0, 25.5)],
 [(13.0, 27.5),
  (10.5, 20.5),
  (8.5, 21.5),
  (22.0, 

Creamos la matriz de distancias

In [ ]:
c = distance_matrix(points, points)  # Distancia entre las ciudades
c[:5, :5]

array([[ 0.        , 14.86606875, 15.        , 18.02775638, 16.        ],
       [14.86606875,  0.        ,  4.47213595, 26.41968963, 25.23885893],
       [15.        ,  4.47213595,  0.        , 29.15475947, 27.73084925],
       [18.02775638, 26.41968963, 29.15475947,  0.        ,  2.23606798],
       [16.        , 25.23885893, 27.73084925,  2.23606798,  0.        ]])

## Visualización de los datos

In [ ]:
plot_cities_and_route(points, product(range(n), range(n))) # Visualización de las ciudades

6 -> 5 -> 2 -> 1 -> ....

# TSP con programación lineal

Considere $n$ puntos, $V = \left\{0, 1, \dots, n-1 \right\}$, y la matriz de distancia $D_{n\times n}$ con entradas $c_{i, j} \in \mathbb{R^+}$.

La variable $x_{i,j}$ es tal que 
$$  x_{i,j} = \begin{cases} 1 & \text{ el camino va de la ciudad } i \text{ a la ciudad } j\text{,} \\ 0 & \text{caso contrario.} \end{cases} $$

Tomando una variable dummy $y_i$ que guarda información del orden en que se visitan las ciudades, a partir de la ciudad $1$. Esto se escribe como $ y_i < y_j$ si la ciudad $i$ se visita antes que la ciudad $j$.

Programación lineal favorece la desigualdades no-estrictas respecto a la estrictas, por lo que se puede imponer una condición parecida a la siguiente

$$ y_j \geqslant y_i + x_{i,j} $$ 

cuando $x_{i,j} = 1$ (NOTESE que no es lo mismo que tomar solo $ y_j \geqslant y_i + x_{i,j}$)


La solución es un conjunto de $n$ pares de puntos indicando la ciudad de salida y ciudad de llegada. Considerando las restricciones que se mencionaron al inicio, tenemos que

Minimizar:
$$\sum_{i\in V, j\in V} c_{i, j}x_{i, j}$$
Sujeto a:
\begin{align}
\sum_{i\in V\setminus \{j\}} x_{i, j} = 1 && \forall j \in V \\
\sum_{j\in V\setminus \{i\}} x_{i, j} = 1 && \forall i \in V \\
y_j - (n+1)x_{i, j} \geq y_i - n && \forall i \in V \setminus \{0\}, j\in V\setminus \{0, i\} \\
x_{i, j} \in \{0, 1\} && \forall i\in V, j \in V \\
y_i \geq 0 && \forall i \in V
\end{align}

In [ ]:
# Número de nodos y vértices
n, V = len(points), set(range(len(points)))
print(f'Número de nodos:\tn = {n}\nVértices:\tV = {V}')

Número de nodos:	n = 10
Vértices:	V = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [ ]:
model = Model()

# Variables binarias que indican si se toma el camino de la ciudad i a la j
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# Variables continuas para evitar subrutas
y = [model.add_var() for i in V]

$$\sum_{i\in V, j\in V} c_{i, j}x_{i, j}$$


In [ ]:
# Función objetivo: 
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

$$\sum_{j\in V\setminus \{i\}} x_{i, j} = 1 \text{, } \forall i \in V$$

In [ ]:
# Restricción : Sal de cada ciudad una única vez
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

$$\sum_{i\in V\setminus \{j\}} x_{i, j} = 1 \text{, } \forall j \in V$$

In [ ]:
# Restricción : Entra a cada ciudad una única vez
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

$$y_j - (n+1)x_{i, j} \geq y_i - n \text{, } \forall i \in V \setminus \{0\}, j\in V\setminus \{0, i\}$$

In [ ]:
# Elimina subrutas
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[j] - (n+1)*x[i][j] >= y[i]-n

In [ ]:
# Optimizar
model.optimize()

# Revisar si se encontró una solución
edges = []
if model.num_solutions:
    print('Ruta con distancia total %g encontrada: %s'
              % (model.objective_value, points.index[0]))
    nc = 0 # Indice de la ciudad actual
    while True:
        oc = nc 
        nc = [i for i in V if x[nc][i].x >= 0.99][0] # Indice de la ciudad a la que se siguió
        edges.append((oc, nc)) # Guardamos el camino a tomar
        print(' -> %s' % points.index[nc]) # Imprimimos el siguiente paso de la ruta
        if nc == 0:  # Si regresa al inicio terminamos el ciclo
            break
    print('\n')

Ruta con distancia total 82.3372 encontrada: ciudad_0
 -> ciudad_9
 -> ciudad_1
 -> ciudad_2
 -> ciudad_8
 -> ciudad_7
 -> ciudad_6
 -> ciudad_3
 -> ciudad_4
 -> ciudad_5
 -> ciudad_0




In [ ]:
plot_cities_and_route(points, edges)